In [1]:
import torch

class LinearNoiseScheduler():
    def __init__(self, num_steps, beta_start, beta_end):
        self.num_steps = num_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.step = 0

        # pre-compute alphas and betas
        self.betas = torch.linspace(beta_start, beta_end, num_steps)
        self.alphas = 1 - self.betas
        # \bar{\alpha}_t}
        self.alpha_cum_prod = torch.cumprod(self.alphas, 0)
        # \sqrt{\bar{\alpha}_t}}
        self.sqrt_alpha_cum_prod = torch.sqrt(self.alpha_cum_prod)
        # \sqrt{1-\bar{\alpha}_t}}
        self.sqrt_one_minus_alpha_cum_prod = torch.sqrt(1 - self.alpha_cum_prod)

    # forward process
    def add_noise(self, original, noise, t):
        original_shape = original.shape
        batch_size = original_shape[0]

        sqrt_alph_cum_prod = self.sqrt_alpha_cum_prod[t].repeat(batch_size, 1)
        sqrt_one_minus_alpha_cum_prod = self.sqrt_one_minus_alpha_cum_prod[t].repeat(batch_size, 1)

        for _ in range(original.dim() - 1):
            sqrt_alph_cum_prod = sqrt_alph_cum_prod.unsqueeze(-1)
            sqrt_one_minus_alpha_cum_prod = sqrt_one_minus_alpha_cum_prod.unsqueeze(-1)

        # \sqrt{\bar{\alpha}_t}} * x_0 + (1-\sqrt{\bar{\alpha}_t}) * \epsilon_t
        return sqrt_alph_cum_prod * original + sqrt_one_minus_alpha_cum_prod * noise
    
    def sample_prev_timestep(self, xt, noise_pred, t):
        # x0 = (xt - \sqrt{1-\bar{\alpha}_t}} * \epsilon_t) / \sqrt{\bar{\alpha}_t}}
        x0 = (
            xt - self.sqrt_one_minus_alpha_cum_prod[t] * noise_pred
        ) / self.sqrt_alpha_cum_prod[t]

        x0 = torch.clamp(x0, -1, 1)

        mean = xt - (self.betas[t] * noise_pred) / self.sqrt_one_minus_alpha_cum_prod[t]
        mean = mean / torch.sqrt(self.alphas[t])

        if t == 0:
            return mean, x0

        variance = (1 - self.alpha_cum_prod[t-1]) / (1 - self.alpha_cum_prod[t])
        variance *= self.betas[t]
        sigma = torch.sqrt(variance)
        # sample from Gaussian distribution
        z = torch.randn(xt.shape).to(xt.device)
        return mean + sigma * z, x0

## UNet model

 * Using sinusoidal position embedding for time-embeddings

$$sin\left(pos / 10000^{2i / d_{model}}\right)$$
$$cos\left(pos / 10000^{2i+1 / d_{model}}\right)$$

In [2]:
import torch
import torch.nn as nn

def get_time_embedding(time_steps, t_emb_dim):
    device = time_steps.device
    factor = 10000 ** ((
        torch.arange(0, t_emb_dim//2, device=device) / (t_emb_dim // 2)
    ))

    t_emb = time_steps.unsqueeze(-1).repeat(1, t_emb_dim//2) / factor
    t_emb = torch.cat([torch.sin(t_emb), torch.cos(t_emb)], dim=-1)

    return t_emb
